In [1]:
import psycopg2
import csv

In [1]:
import psycopg2
conn = psycopg2.connect(dbname="postgres", user="postgres", password = "abc123")
cursor = conn.cursor()
cursor.execute("CREATE TABLE notes(id integer PRIMARY KEY, body text, title text)")
conn.close()

In [3]:
conn = psycopg2.connect("dbname=postgres user=postgres")
cur = conn.cursor()
cur.execute("""
    CREATE TABLE users (id integer PRIMARY KEY, 
                        email text , 
                        name text , 
                        address text);""")
with open('user_accounts.csv','r') as file:
    next(file)
    reader = csv.reader(file)
    for row in reader:
        cur.execute("INSERT INTO users VALUES (%s,%s,%s,%s);",row)
conn.commit()
conn.close()

OperationalError: could not connect to server: Connection refused (0x0000274D/10061)
	Is the server running on host "localhost" (::1) and accepting
	TCP/IP connections on port 5432?
could not connect to server: Connection refused (0x0000274D/10061)
	Is the server running on host "localhost" (127.0.0.1) and accepting
	TCP/IP connections on port 5432?


In [ ]:
print(cur.description)

In [ ]:
cur.execute("""ALTER TABLE ign_reviews ALTER COLUMN id TYPE int8;""")

In [ ]:
# tao column chi co cac value di kem
cur.execute("""
    CREATE TYPE evaluation_enum AS ENUM (
    'Great',       'Mediocre', 'Bad', 
    'Good',        'Awful',    'Okay', 
    'Masterpiece', 'Amazing',  'Unbearable', 
    'Disaster',    'Painful');
""")

cur.execute("""ALTER TABLE ign_reviews
ALTER COLUMN score_phrase TYPE evaluation_enum
USING score_phrase::evaluation_enum;""")

In [ ]:
cur.execute("""
    ALTER TABLE ign_reviews 
    ALTER COLUMN editors_choice TYPE boolean
    USING editors_choice::boolean;
""")

In [ ]:
cur.execute("""ALTER TABLE ign_reviews DROP COLUMN release_day;""")

In [ ]:
# 3 cot cuoi la ngay, thang, nam -> 1 cot date va cho vao table
with open('ign.csv') as file:
    next(file)
    reader = csv.reader(file)
    for row in reader:
        year = int(row[8])
        month = int(row[9])
        day = int(row[10])
        date = datetime.date(year,month,day)
        row = row[:-3] # chi lay cac cot con lai tru 3 cot cuoi
        row.append(date) # tao them cot date
        cur.execute("""INSERT INTO ign_reviews VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s);""",row)

In [ ]:
# Insert bang dictionary :
row_values = { 
    'identifier': 1, 
    'mail': 'adam.smith@dataquest.io',
    'name': 'Adam Smith', 
    'address': '42 Fake Street'
}
cur.execute("INSERT INTO users VALUES (%(identifier)s,%(mail)s,%(name)s,%(address)s);",row_values)

In [ ]:
# SQL injection
def get_email(name):
    import psycopg2
    conn = psycopg2.connect("dbname=dq user=dq")
    cur = conn.cursor()
    # create the query string using the format function
    query_string = "SELECT email FROM users WHERE name = '" + name + "';"
    # execute the query
    cur.execute(query_string)
    res = cur.fetchall()
    conn.close()
    return res

name = "Joseph Kirby' OR 1 = 1 --"
name = "Larry Cain' UNION SELECT address FROM users WHERE name = 'Larry Cain"
# Query all information from interface
# Fix it:
def get_email_fixed(name):
   import psycopg2
   conn = psycopg2.connect("dbname=dq user=dq")
   cur = conn.cursor()
   cur.execute("SELECT email FROM users WHERE name = %s",(name,))
   res = cur.fetchall()
   conn.close()
   return res

# THE BEST THING IN SQL is PREPARE - giong Function in Python va no con accelerate system faster:
cur.execute("""PREPARE get_email(text) AS SELECT email FROM users WHERE name = $1;""")
name = 'Anna Carter'
cur.execute("""EXECUTE get_email(%s)""",(name,))

In [ ]:
# Encoding from Byte to String:
game_data = (52499790661213, 'Amazing', 'LittleBigPlanet PS Vita', '/games/littlebigplanet-vita/vita-98907', 'PlayStation Vita', 9.0, 'Platformer', 'y', date(2012, 12, 9))
conn_encoding = conn.encoding
mogrified_values = cur.mogrify("(%s, %s, %s, %s, %s, %s, %s, %s, %s)",game_data)
mogrified_values_decoded = mogrified_values.decode(conn_encoding)

In [ ]:
with open("temp.csv","w") as file:
    cur.copy_expert("COPY ign_reviews TO STDOUT WITH CSV HEADER;",file)

# Nhap database vao hoac xuat ra khoi file:

with open("temp.csv","r") as file:
    cur.copy_expert("COPY ign_reviews_copy FROM STDIN WITH CSV HEADER;",file)

In [ ]:
# Chuyen Data tu DB nay sang DB khac:
cur.execute("CREATE TABLE ign_restricted (id bigint PRIMARY KEY, title varchar(200), release_date date);")
cur.execute("INSERT INTO ign_restricted (id, title, release_date) SELECT id, title, release_date FROM ign_reviews;")